In [4]:
import numpy as np
from sympy import *
from sympy.matrices import Matrix

Define Testdata

In [5]:
test_set_1 = { "xpos" : 2.3029, "ypos" : 0, "zpos" : 1.9466, "roll" : 0, "pitch" : 0, "yaw" : 0 , 
               "theta1_rviz" : 0, "theta2_rviz" : 0, "theta3_rviz" : 0, 
               "theta4_rviz" : 0, "theta5_rviz" : 0, "theta6_rviz" : 0,
               "wcx_rviz" : 1.84990000000000, "wcy_rviz" : 0, "wcz_rviz" : 1.94660000000000,}
test_set_2 = { "xpos" : 2.153, "ypos" : 0, "zpos" : 1.9466, "roll" : 0, "pitch" : 0, "yaw" : 0 , 
               "theta1_rviz" : 0, "theta2_rviz" : 0, "theta3_rviz" : 0, 
               "theta4_rviz" : 0, "theta5_rviz" : 0, "theta6_rviz" : 0,}
test_set_3 = { "xpos" : 1.801, "ypos" : 1.005, "zpos" : 1.148, "roll" : 1.557, "pitch" : 0.683, "yaw" : 2.649 , 
               "theta1_rviz" : 0.4, "theta2_rviz" : 0.2, "theta3_rviz" : 0.2, 
               "theta4_rviz" : 1, "theta5_rviz" : 1, "theta6_rviz" : 1,
               "wcx_rviz" : 1.84990000000000, "wcy_rviz" : 0, "wcz_rviz" : 1.94660000000000,}
test_set_3 = { "xpos" : 1.801, "ypos" : 1.005, "zpos" : 1.148, "roll" : 1.557, "pitch" : 0.683, "yaw" : 2.649 , 
               "theta1_rviz" : 0.72, "theta2_rviz" : 0.73, "theta3_rviz" : -0.93, 
               "theta4_rviz" : 1, "theta5_rviz" : -0.37, "theta6_rviz" : -1.09,
               "wcx_rviz" : 1.84990000000000, "wcy_rviz" : 0, "wcz_rviz" : 1.94660000000000,}

# Choose current set of test data
current_test_set = test_set_3

First create the symbols

In [7]:
# Create symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

Then create modified DH parameters

In [11]:
### KUKA KR210 ###
# Create Modified DH parameters
s = {alpha0:     0,  a0:      0, d1:  0.75,  
     alpha1: -pi/2,  a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0,       
     alpha3: -pi/2,  a3: -0.054, d4:  1.50,
     alpha4:  pi/2,  a4:      0, d5:     0,
     alpha5: -pi/2,  a5:      0, d6:     0,
     alpha6:     0,  a6:      0, d7: 0.303, q7: 0}

print(alpha0.evalf(subs=s))

0


Define the transformation matrix

In [151]:
# Define Modified DH Transformation matrix
def build_transformation_matrix( theta, alpha, a, d ):
    T = Matrix([[            cos(theta),           -sin(theta),           0,             a ],
                [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d ],
                [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d ],
                [                     0,                     0,           0,             1 ]]) 
    T = T.subs(s)
       
    return(T)

# Modified DH params
T0_1 = build_transformation_matrix( q1, alpha0, a0, d1 )
T1_2 = build_transformation_matrix( q2, alpha1, a1, d2 )
T2_3 = build_transformation_matrix( q3, alpha2, a2, d3 )
T3_4 = build_transformation_matrix( q4, alpha3, a3, d4 )
T5_5 = build_transformation_matrix( q5, alpha5, a5, d5 )
T5_6 = build_transformation_matrix( q6, alpha5, a5, d6 )
T6_G = build_transformation_matrix( q7, alpha6, a6, d7 )


Correct the Orientation difference between the definition of the gripper_link in the urdf-file and the Denavit-Hardenberg Convention.

First apply a body-fixed rotation about the z-axis followed by a body-fixed rotation about the y-axis.

R_corr is the composition of those two rotations.

In [152]:
#Correct the Gripper
R_z =  Matrix([[     cos(np.pi), -sin(np.pi),             0, 0],
               [     sin(np.pi),  cos(np.pi),             0, 0],
               [              0,           0,             1, 0],
               [              0,           0,             0, 1]])

R_y =  Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2), 0],
               [              0,           1,             0, 0],
               [ -sin(-np.pi/2),           0, cos(-np.pi/2), 0],
               [              0,           0,             0, 1]])

R_corr = simplify(R_z * R_y)

Create individual transformation matrices

In [153]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to gripper_link
T0_G_corr = simplify(T0_G * R_corr) # Total HT between base_link and gripper_link with orientation correction applied

#T0_G = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G)
sub_dict = { q1: -2.77, q2: 0.36, q3: -2.41, q4: 4.56, q5: 1.31, q6: 4.70 }
sub_dict_zero = { q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0 }
# Evaluate transforms
print ("T0_1 = ", T0_1.evalf(subs=sub_dict_zero),"\n")
print ("T0_2 = ", T0_2.evalf(subs=sub_dict_zero),"\n")
print ("T0_3 = ", T0_3.evalf(subs=sub_dict_zero),"\n")
print ("T0_4 = ", T0_4.evalf(subs=sub_dict_zero),"\n")
print ("T0_5 = ", T0_5.evalf(subs=sub_dict_zero),"\n")
print ("T0_6 = ", T0_6.evalf(subs=sub_dict_zero),"\n")
print ("T0_G = ", T0_G.evalf(subs=sub_dict_zero),"\n")
print  ("T0_G_corr = ", T0_G_corr.evalf(subs=sub_dict_zero),"\n")
#print ("T0_1 = ", T0_1,"\n")
#print ("T0_2 = ", T0_2,"\n")
#print ("T0_3 = ", T0_3,"\n")
#print ("T0_4 = ", T0_4,"\n")
#print ("T0_5 = ", T0_5,"\n")
#print ("T0_6 = ", T0_6,"\n")
#print ("T0_G = ", T0_G,"\n")
#print  ("T0_G_corr = ", T0_G_corr,"\n")

T0_1 =  Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]]) 

T0_2 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0, 0.75],
[  0,   0,   0,  1.0]]) 

T0_3 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0,  2.0],
[  0,   0,   0,  1.0]]) 

T0_4 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_5 =  Matrix([
[  0, 1.0,   0,  1.85],
[  0,   0, 1.0,     0],
[1.0,   0,   0, 1.946],
[  0,   0,   0,   1.0]]) 

T0_6 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_G =  Matrix([
[  0,    0, 1.0, 2.153],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_G_corr =  Matrix([
[                  1.0,                     0, 6.12323399573677e-17, 2.153],
[-7.49879891330929e-33,                   1.0, 1.2

Extract End-Effector position and orient from request
Calculate joint angles using Geometric IK method

In [154]:
# Build a rotation matrix from the Roll, Pitch and Yaw angles
def get_wrist_rot_matrix(roll, pitch, yaw):
        
    # Rrpy = Rx * Ry * Rz
    Rrpy = Matrix([[    cos(yaw)*cos(pitch),   cos(yaw)*sin(pitch)*sin(roll) - sin(yaw)*cos(roll),    cos(yaw)*sin(pitch)*cos(roll) + sin(yaw)*sin(roll)],
                   [    sin(yaw)*cos(pitch),   sin(yaw)*sin(pitch)*sin(roll) + cos(yaw)*cos(roll),    sin(yaw)*sin(pitch)*cos(roll) - cos(yaw)*sin(roll)],
                   [            -sin(pitch),             cos(pitch)*sin(roll),                                       cos(pitch)*cos(roll)               ]])

    print("Rrpy", Rrpy)
    return Rrpy

Find the location of the WC relative to the base frame

In [206]:
def get_wrist_pos(px, py, pz, Rrpy):
           
    end_effector_length = 0.453
    #end_effector_length = 0.303
    
    
    lx = Rrpy[ 0, 0 ]
    ly = Rrpy[ 1, 0 ]
    lz = Rrpy[ 2, 0 ]

    print("lx:",lx," ly:",ly," lz:",lz)

    d6 = 0

    # Calculate Wrist Center
    wx = px - ( end_effector_length + d6 ) * lx
    wy = py - ( end_effector_length + d6 ) * ly
    wz = pz - ( end_effector_length + d6 ) * lz
    
    # Modify Rrpy to take in transformation from gripper
    Rrpy = Rrpy.row_join(Matrix([[px],[py],[pz]]))
    Rrpy = Rrpy.col_join(Matrix([[0,0,0,1]]))

    print("wx:",wx," wy:",wy," wz:",wz)
    print("Rrpy:", Rrpy)
    return wx, wy, wz, Rrpy


In [193]:
Rrpy = get_wrist_rot_matrix(test_roll, test_pitch, test_yaw)
wx, wy, wz, Rrpy = get_wrist_pos(test_pos_x, test_pos_y, test_pos_z, Rrpy)

test_roll = 0
test_pitch = 0
test_yaw = 0
test_pos_x = 2.3029
test_pos_y = 0
test_pos_z = 1.9466

Rrpy Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
lx: 1  ly: 0  lz: 0
wx: 1.84990000000000  wy: 0  wz: 1.94660000000000
Rrpy: Matrix([[1, 0, 0, 2.30290000000000], [0, 1, 0, 0], [0, 0, 1, 1.94660000000000], [0, 0, 0, 1]])


find joint variables, q1, q2 and q3, such that the WC has coordinates equal to equation (3). This is the hard step. One way to attack the problem is by repeatedly projecting links onto planes and using trigonometry to solve for joint angles. Unfortunately, there is no generic recipe that works for all manipulators so you will have to experiment.

In [224]:
# Calculate joint angles using Geometric IK method
def get_theta_123 (wx, wy, wz):
    
    theta1 = atan2(wy, wx)
    a_1 = 0.35
    a_2 = 1.25
    d_1 = 0.75
    d_4 = 1.50
    a_3 = -0.054
    
    x_c = sqrt( wx**2 + wy**2 ) - a_1 # Subtract a1 as horizontal offset from robot-base
    z_c = wz - d_1 # Subtract d1 as vertical offset from robot base

    L_25 = sqrt(x_c**2 + z_c**2)
    L_35 = sqrt(a_3**2+d_4**2)

    cos_theta_3 = ( L_25**2 - a_2**2 - L_35**2) / ( 2 * a_2 * L_35 )
    #theta3 = atan2( cos_theta_3, sqrt( 1 - cos_theta_3**2 ))
    q3 = atan2( sqrt( 1 - cos_theta_3**2 ), cos_theta_3 )
    theta3 = ( q3 - np.pi/2 ).evalf()
    print("theta3: ", theta3)
    theta3_alt = ( -q3 + np.pi/2 ).evalf()
    print("theta3 alternative:", theta3_alt)
    
    # theta2 = atan2( z_c, x_c ) - atan2(a_2*sin(theta3), a_2+L_35*cos(theta3))
    # theta2b = atan2( z_c, x_c ) - atan2(a_2*sin(theta3b), a_2+L_35*cos(theta3b))
    beta_2 = atan2( z_c, x_c )
    cos_beta_1 = ( ( -L_35**2 + L_25**2 + a_2**2 ) / ( 2 * L_25 * a_2 ) )
    beta_1 = atan2(  cos_beta_1, sqrt( 1 - cos_beta_1**2 ) ) 
    theta2 = np.pi/2 - beta_1 - beta_2

    print("theta1,2,3:",theta1, theta2, theta3)
        
    return theta1.evalf(), theta2.evalf(), theta3.evalf()

In [181]:
#theta1, theta2, theta3 = get_theta_123( test_wx, test_wy, test_wz )

once the first three joint variables are known, calculate 03R via application of homogeneous transforms up to the WC.

In [177]:
def transform_to_wc(theta1, theta2, theta3, Rrpy):
 
    R0_3 = simplify( T0_1 * T1_2 * T2_3 )
    #R0_3 = Matrix([[    sin(theta2 + theta3)*cos(theta1),   cos(theta1)*cos(theta2 + theta3),   -sin(theta1)],
                   #[   sin(theta1)*sin(theta2 + theta3),   sin(theta1)*cos(theta2 + theta3),   cos(theta1)],
                   #[   cos(theta2 + theta3),           -sin(theta2 + theta3),          0]])
    print("R0_3:", R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3}))
    R0_3 = R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
    R3_6_eval = R0_3.inv() * Rrpy
    R3_6_var = simplify( T3_4 * T4_5 * T5_6 )
    print("R3_6:", R3_6_var)
    return R3_6_var, R3_6_eval        

In [182]:
#R3_6_var, R3_6_eval = transform_to_wc( theta1, theta2, theta3, Rrpy )

find a set of Euler angles corresponding to the rotation matrix

In [173]:
def get_theta_456(R3_6_var, R3_6_eval):
    r13 = R3_6_eval[0,2]
    r33 = R3_6_eval[2,2]
    r23 = R3_6_eval[1,2]
    r22 = R3_6_eval[1,1]
    r21 = R3_6_eval[1,0]
    
    print("r13: ", R3_6_var[0,2])
    print("r23: ", R3_6_var[1,2])
    print("r33: ", R3_6_var[2,2])    
    sin_q5_var = sqrt(R3_6_var[0,2]**2 + R3_6_var[2,2]**2)
    print("sin(q5): ", sin_q5_var)
    # sin(q5)² * ( cos(q4)² + sin(q4)²)
    # cos(q4)² + sin(q4)² = 1
    # => sqrt(r13² + r33²) = sin(q5)
    sin_q5 = sqrt(r13**2 + r33**2).evalf()
           
    theta5 = atan2( sin_q5, r23 ).evalf()
    
    if( sin_q5 < 0 ):
        theta4 = atan2(-r33,  r13).evalf()
        theta6 = atan2( r22, -r21).evalf()
    else:
        theta4 = atan2( r33,  -r13).evalf()
        theta6 = atan2( -r22, r21).evalf()
    
    print("theta4, theta5, theta6: ",theta4, theta5, theta6)

    
    return theta4, theta5, theta6



r13:  -sin(q5)*cos(q4)
r23:  cos(q5)
r33:  sin(q4)*sin(q5)
sin(q5):  sqrt(sin(q4)**2*sin(q5)**2 + sin(q5)**2*cos(q4)**2)
theta4, theta5, theta6:  0 2.71688678075291 3.14159265358979


In [183]:
#theta4, theta5, theta6 = get_theta_456( R3_6_var, R3_6_eval )

In [228]:
Rrpy                                = get_wrist_rot_matrix(current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"])
test_wx,  test_wy,    test_wz, Rrpy = get_wrist_pos(current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"], Rrpy)
theta1,   theta2,     theta3        = get_theta_123( test_wx, test_wy, test_wz )
R3_6_var, R3_6_eval                 = transform_to_wc( theta1, theta2, theta3, Rrpy )
theta4,   theta5,     theta6        = get_theta_456( R3_6_var, R3_6_eval )

Rrpy Matrix([[-0.683461696574014, -0.562559850892265, 0.465195145589723], [0.366829637452813, 0.286281429692164, 0.885143468653330], [-0.631122909109160, 0.775609023398699, 0.0107012345132777]])
lx: -0.683461696574014  ly: 0.366829637452813  lz: -0.631122909109160
wx: 2.11060814854803  wy: 0.838826174233876  wz: 1.43389867782645
Rrpy: Matrix([[-0.683461696574014, -0.562559850892265, 0.465195145589723, 1.80100000000000], [0.366829637452813, 0.286281429692164, 0.885143468653330, 1.00500000000000], [-0.631122909109160, 0.775609023398699, 0.0107012345132777, 1.14800000000000], [0, 0, 0, 1]])
theta3:  -0.0916063373621652
theta3 alternative: 0.0916063373621652
theta1,2,3: 0.378291835266922 0.480647673672053 -0.0916063373621652
R0_3: Matrix([
[0.352483757447434,  0.859853406220867, -0.369333616864361, 0.862333555317037],
[0.140088818449136,  0.341734463423921,  0.929296873692088, 0.342720157513704],
[0.925273107618104, -0.379301563823215,                  0,  1.85836956818382],
[             

Test your IK based on your FK

In [229]:
print("---------------- input data ----------------")
print("px, py, pz:   ", current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"])
print("r , p , y :   ", current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"],"\n")
print("--------------- wrist center ---------------")
print("WC_x: ", test_wx, "expected: ", current_test_set["wcx_rviz"])
print("WC_y: ", test_wy, "expected: ", current_test_set["wcy_rviz"])
print("WC_z: ", test_wz, "expected: ", current_test_set["wcz_rviz"],"\n")
print("--------------- joint angles ---------------")
print("theta 1:", theta1, "expected: ", current_test_set["theta1_rviz"])
print("theta 2:", theta2, "expected: ", current_test_set["theta2_rviz"])
print("theta 3:", theta3, "expected: ", current_test_set["theta3_rviz"])
print("theta 4:", theta4, "expected: ", current_test_set["theta4_rviz"])
print("theta 5:", theta5, "expected: ", current_test_set["theta5_rviz"])
print("theta 6:", theta6, "expected: ", current_test_set["theta6_rviz"],"\n")
print("--------------- final matrices ---------------")
theta_values = { q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6 }
print("R0_6: ", T0_6.evalf(subs=theta_values))
print("Rrpy: ", Rrpy.evalf())

---------------- input data ----------------
px, py, pz:    1.801 1.005 1.148
r , p , y :    1.557 0.683 2.649 

--------------- wrist center ---------------
WC_x:  2.11060814854803 expected:  1.8499
WC_y:  0.838826174233876 expected:  0
WC_z:  1.43389867782645 expected:  1.9466 

--------------- joint angles ---------------
theta 1: 0.378291835266922 expected:  0.4
theta 2: 0.480647673672053 expected:  0.2
theta 3: -0.0916063373621652 expected:  0.2
theta 4: 2.00006844562338 expected:  1
theta 5: 0.797602364144892 expected:  1
theta 6: 1.88756336762247 expected:  1 

--------------- final matrices ---------------
R0_6:  Matrix([
[-0.683461696574014, -0.562559850892265,  0.465195145589723,  2.13307954174618],
[ 0.366829637452813,  0.286281429692164,   0.88514346865333, 0.847757056453333],
[ -0.63112290910916,  0.775609023398699, 0.0107012345132779,  1.23945247463762],
[                 0,                  0,                  0,               1.0]])
Rrpy:  Matrix([[-0.683461696574014, -